#  Assignment 4 - Fairness

## Contributers:


#### Reni Koci (renik98) 
#### Viktoria Voucheva (vouchev00)
#### Muhammad Furqan Rafique (rafim96)

In this assignment we will be looking at the AI Fairness 360 toolkit. It is an "extensible open-source library containing techniques developed by the research community to help detect and mitigate bias in machine learning models throughout the AI application lifecycle. AI Fairness 360 package includes multiple methods for computing different fairness metrics and algorithms for bias mitigation. In this task, you will get familiar with AIF360. You will use it to try to detect and mitigate bias on a self-chosen dataset.

## <center>1. AIF360 Installation Instructions (Ubuntu) </center> 

Execute following cmd's for setup 

1. conda deactivate (OR deactivate)
2. sudo pip3 install virtualenv 
3. sudo add-apt-repository ppa:deadsnakes/ppa
4. sudo apt install python3.7
5. sudo apt-get install python3.7-distutils
6. virtualenv -p /usr/bin/python3.7 venv
7. source venv/bin/activate
8. pip3 install –upgrade pip
9. pip install aif360


10. python -m notebook        (Open the notebook using this cmd and start working)


## <center> 4 & 5. Choosing datasets and a protected attribute </center> 

importing libraries

In [2]:
import numpy as np
from aif360.datasets import BankDataset
import warnings
warnings.filterwarnings('ignore')

We are selecting BankDataset. And we are using AGE (age >=25 is considered privileged) as protected attribute. This dataset also contains protected attribute like "marital" which we do not consider in this evaluation

In [3]:
dataset_orig = BankDataset(
    protected_attribute_names=['age'],           
    privileged_classes=[lambda x: x >= 25],       
    features_to_drop=['marital']                 
)


In [4]:
dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True) # split into train(70%)-test(30%)

In [5]:
privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

## <center> 6. Compute multiple fairness metrics  </center> 

#### Here, we'll use the metrics from BinaryLabelDatasetMetric class

In [6]:
from aif360.metrics import BinaryLabelDatasetMetric

In [7]:
binary_metric = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
print("_________Metrics on Original training dataset__________")
print("Mean difference                             = %f" % binary_metric.mean_difference())
print("Base rate                                   = %f" % binary_metric.base_rate())
print("Disparate impact                            = %f" % binary_metric.disparate_impact())
print("smoothed_empirical_differential_fairness    = %f" % binary_metric.smoothed_empirical_differential_fairness())


_________Metrics on Original training dataset__________
Mean difference                             = 0.108291
Base rate                                   = 0.127210
Disparate impact                            = 1.872499
smoothed_empirical_differential_fairness    = 0.629008


## <center> 7,8 & 9. Mitigate bias using bias mitigation algorithm plus recompute and compare the fairness metrics  </center> 

For this part we'll use different bias mitigation algorithms and will see their impact on fairness metrics

### 1. Reweighing

In [8]:
from aif360.algorithms.preprocessing import Reweighing
RW = Reweighing(unprivileged_groups=unprivileged_groups,privileged_groups=privileged_groups)
RW_transf_train = RW.fit_transform(dataset_orig_train)

In [9]:
binary_metric = BinaryLabelDatasetMetric(RW_transf_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
print("_________Metrics on Reweighing Transform data__________")
print("Mean difference                             = %f" % binary_metric.mean_difference())
print("Base rate                                   = %f" % binary_metric.base_rate())
print("Disparate impact                            = %f" % binary_metric.disparate_impact())
print("smoothed_empirical_differential_fairness    = %f" % binary_metric.smoothed_empirical_differential_fairness())


_________Metrics on Reweighing Transform data__________
Mean difference                             = -0.000000
Base rate                                   = 0.127210
Disparate impact                            = 1.000000
smoothed_empirical_differential_fairness    = 0.004636


### 2. Disparate Impact Remover

In [10]:
from aif360.algorithms.preprocessing import DisparateImpactRemover
dis_imp_rem =DisparateImpactRemover(repair_level=0.50, sensitive_attribute='age')
dis_imp_rem_transf_train = dis_imp_rem.fit_transform(dataset_orig_train)

In [11]:
binary_metric = BinaryLabelDatasetMetric(dis_imp_rem_transf_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
print("_________Metrics on Disparate Impact Remover Transform data__________")
print("Mean difference                             = %f" % binary_metric.mean_difference())
print("Base rate                                   = %f" % binary_metric.base_rate())
print("Disparate impact                            = %f" % binary_metric.disparate_impact())
print("smoothed_empirical_differential_fairness    = %f" % binary_metric.smoothed_empirical_differential_fairness())


_________Metrics on Disparate Impact Remover Transform data__________
Mean difference                             = 0.108291
Base rate                                   = 0.127210
Disparate impact                            = 1.872499
smoothed_empirical_differential_fairness    = 0.629008


### 3. LFR

In [12]:
from aif360.algorithms.preprocessing import LFR
lfr =LFR(unprivileged_groups=unprivileged_groups,privileged_groups=privileged_groups)
lfr_transf_train = lfr.fit_transform(dataset_orig_train)

In [13]:
binary_metric = BinaryLabelDatasetMetric(lfr_transf_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
print("_________Metrics on LFR Transform data__________")
print("Mean difference                             = %f" % binary_metric.mean_difference())
print("Base rate                                   = %f" % binary_metric.base_rate())
print("Disparate impact                            = %f" % binary_metric.disparate_impact())
print("smoothed_empirical_differential_fairness    = %f" % binary_metric.smoothed_empirical_differential_fairness())


_________Metrics on LFR Transform data__________
Mean difference                             = 0.000000
Base rate                                   = 0.000000
Disparate impact                            = nan
smoothed_empirical_differential_fairness    = 3.524629


### 4. MetaFairClassifier

In [14]:
from aif360.algorithms.inprocessing import MetaFairClassifier
mfc = MetaFairClassifier(sensitive_attr='age')

In [15]:
# pip install fairlearn # if you get error while running above commands

In [16]:
x = mfc.fit(dataset_orig_train)
mfc_transf_train = x.predict(dataset_orig_train)

In [17]:
binary_metric = BinaryLabelDatasetMetric(mfc_transf_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
print("_________Metrics on MetaFairClassifier Transform data__________")
print("Mean difference                             = %f" % binary_metric.mean_difference())
print("Base rate                                   = %f" % binary_metric.base_rate())
print("Disparate impact                            = %f" % binary_metric.disparate_impact())
print("smoothed_empirical_differential_fairness    = %f" % binary_metric.smoothed_empirical_differential_fairness())


_________Metrics on MetaFairClassifier Transform data__________
Mean difference                             = 0.342318
Base rate                                   = 0.240296
Disparate impact                            = 2.485021
smoothed_empirical_differential_fairness    = 0.910017


#### Types of bias mitigation algorithms:
There are mainly three types of bias mitigation algorithms:
1. pre-processing
2. in-processing
3. post-processing

Pre-processing algorithms are used to mitigate the biasness before the model training. This category has the following algorithms:
1. Reweighing: only changes weights applied to training samples; it does not change any feature or label values. Therefore, it may be a preferred option in case the application does not allow for value changes
2. Disparate impact remover: yield modified datasets in the same space as the input training data
3. LFR: the pre-processed dataset is in a latent space. 

Whereas, in in-processing algorithms, the training algorithms are modified to learn fairness while learning predictive model parameters. Examples are :
1. AdversarialDebiasing: Learns a classifier to maximize prediction accuracy and simultaneously reduce an adversary’s ability to determine the protected attribute from the predictions
2. MetaFairClassifier: The meta-algorithm here takes the fairness metric as part of the input and returns a classifier optimized w.r.t. that fairness metric

Examples of post-processing algorithms are :
1. Calibrated equalized odds postprocessing: a post-processing technique that optimizes over calibrated classifier score outputs to find probabilities with which to change output labels with an equalized odds objective
2. Equalized odds postprocessing: a post-processing technique that solves a linear program to find probabilities with which to change output labels to optimize equalized odds

In this assignment we make use of 3 pre-process and 1 in-process bias mitigation algorithm, and our findings are below.

#### Results and Comparison 
In our case, LFR is performing well whereas the in-process algorithm 'MetaFairClassifier' is giving the worst results. Further, we have also found out that each algorithm even within the same category is built for different purposes, and we can not just simply compare all algorithms or metrics as we can do in normal ML models. Moreover, AIF360 has no full support for the dataset that we have chosen for this assignment, otherwise, we would have also implemented post-processing or more in-processing algorithms.

Reference :- https://github.com/Trusted-AI/AIF360

### Fairness Tools Overview:

#### Aequitas
Is an open source bias and fairness audit toolkit that was released in 2018. It is designed to enable developers to seamlessly test models for a series of bias and fairness metrics in relation to multiple population sub-groups.

#### Microsoft Fair Learn
As part of Microsoft Fair Learn, this is a general-purpose methodology for approaching fairness. Using binary classification, the method applies constraints to reduce fair classification to a sequence of cost-sensitive classification problems. Whose solutions yield a randomized classifier with the lowest (empirical) error subject to the desired constraints.

#### What-if Tool from Googl
The What-if Tool from Google is an open-source TensorBoard web application which lets users analyse an ML model without writing code. It visualises counterfactuals so that users can compare a data-point to the most similar point where the model predicts a different result. In addition, users can explore the effects of different classification thresholds, taking into account constraints such as different numerical fairness criteria.  There are a number of demos available – showing how the different functions work on pre-trained models.

Reference:- https://www.aiforpeople.org/tools-for-fairness/